## dataset and dataloader

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
# torch.utils.data.datasets 
from torchvision import datasets # torchvision, torchaudio 등 도메인특화 datasets
from torchvision.transforms import ToTensor
# torchvision.transforms는 데이터 전처리 package로 inference를 위한 현실데이터를 변환하는데도 사용된다.


In [19]:
# sample과 label로 구성되어 있는 데이터셋 로드
# transform을 통해 sample 변환, target_transform을 통해 label변환
# ToTensor로 변환시 sample(img) 차원은 [1,28,28]
train_data = datasets.FashionMNIST(
    root ='./',
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root ='./',
    train = False,
    download= True,
    transform = ToTensor()
)
print(test_data[0])
# 64개씩 묶음
batch_size = 64
# dataloader는 feature와 label로 이루어진 데이터를 64개씩 묶어 iterable한 객체로 반환
# shuffling sampling 등의 기능도 지원
train_loader = DataLoader(train_data, batch_size = batch_size)
test_loader = DataLoader(test_data, batch_size = batch_size)

for data,label in test_loader:
    print(f"tensor data's feature size is:{data.shape}")
    print(f"label tensor size is {label.shape}, dtype is {label.dtype}")
    break


(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

## Model Generate

### tips
> layer의 선형변환은 입력차원을 축소 또는 확장 -> 특징의 밀도를 높이거나 낮춘다 (연산량을 줄이거나 특징을 풍부하게 한다)


> 입력 데이터의 가장 낮은 차원은 batch size값을 가진다. ex) rgb 28*28 batch 3 -> [3,3,28,28]


> flatten을 하여도 dim = 0, 즉 batch 차원은 유지된다. ex) [3,3,28,28] -> [3,784]

> gradient는 매개변수에 대한 손실함수의 변화도이다 --> 이에 따라 매개변수가 조정된다.


> 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다. 비선형 활성화는 선형 변환 후에 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.


> softmax함수의 인자인 dim 은 scale이 조정될 출력벡터 값의 합을 의미한다. ex) nn.Softmax(dim = 1)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device:{device}")

class NeuralNetwork(nn.Module):
    # init에서 신경망 구성요소 정의
    def __init__(self):
        # super().__init__()
        super().__init__()
        self.flatten = nn.Flatten()
        # sequential은 출력벡터를 반환한다
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    # forward 에서 신경망에서 데이터를 처리하는 과정을 지정
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model.to(device)
print(model)

device:cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Loss function & Optimizer

In [4]:
loss_f = nn.CrossEntropyLoss()
criterion = torch.optim.SGD(model.parameters(), lr = 1e-3)
print(len(train_loader.dataset))
print(len(train_loader))


60000
938


## Train & eval

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train(dataloader, model, loss_f, criterion):
    data_size = len(dataloader.dataset)
    for batch_number, (data, label) in enumerate(dataloader):
        data , label = data.to(device), label.to(device)
        prediction = model(data)
        loss = loss_f(prediction, label)

        criterion.zero_grad()   # 초기 gradient를 0으로 만들어준다 (갱신에 영향이 없게)
        loss.backward() # 역전파하며 층별 gradient를 계산
        criterion.step()    # gradient를 가중치에 적용하여 갱신 (gradient descent)

        if batch_number % 100 == 0:
            loss, current = loss.item(), (batch_number+1) * len(data) #item()은 행렬의 원소를 추출 , ex) 100 * 64
            print(f"loss:{loss:.1f}, at data number:{current}/{data_size}")

def test(dataloader, model, loss_f):
    data_size = len(dataloader.dataset)
    batch_number = len(dataloader)
    # model 평가모드
    model.eval()
    loss, correct = 0, 0
    # gradient 변화없이
    with torch.no_grad():
        for data, label in dataloader:
            data , label = data.to(device), label.to(device)
            prediction = model(data)

            loss += loss_f(prediction, label).item()
            correct += (prediction.argmax(1) == label).type(torch.float).sum().item()
    loss /= batch_number
    acc = (correct/data_size)*100
    print(f"test loss avg:{loss:>0.1f} test acc:{acc:.1f}")

## Train session

In [14]:
epochs = 5
for i in range(epochs):
    print(f"{i+1}epoch________________________")
    train(train_loader,model, loss_f, criterion)
    test(test_loader, model, loss_f)

1epoch________________________
loss:1.2, at data number:64/60000
loss:1.2, at data number:6464/60000
loss:1.0, at data number:12864/60000
loss:1.1, at data number:19264/60000
loss:1.0, at data number:25664/60000
loss:1.0, at data number:32064/60000
loss:1.1, at data number:38464/60000
loss:1.0, at data number:44864/60000
loss:1.0, at data number:51264/60000
loss:1.0, at data number:57664/60000
test loss avg:1.0 test acc:65.8
2epoch________________________
loss:1.1, at data number:64/60000
loss:1.1, at data number:6464/60000
loss:0.9, at data number:12864/60000
loss:1.0, at data number:19264/60000
loss:0.9, at data number:25664/60000
loss:0.9, at data number:32064/60000
loss:1.0, at data number:38464/60000
loss:0.9, at data number:44864/60000
loss:1.0, at data number:51264/60000
loss:0.9, at data number:57664/60000
test loss avg:0.9 test acc:67.0
3epoch________________________
loss:1.0, at data number:64/60000
loss:1.0, at data number:6464/60000
loss:0.8, at data number:12864/60000
loss

## Model save & load to Inference

In [20]:
# model의 state_dict(상태 dictionary)를 직렬화하여 저장 (매개변수 등이 포함)
# model구조를 선언한 후 상태 dict를 load
torch.save(model.state_dict(),"torchModel.pth")
print(f"model saved succedssfully")


model saved succedssfully


In [23]:
model = NeuralNetwork()
model.load_state_dict(torch.load("torchModel.pth"))
print(f"model successfully loaded")

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # 평가모드
data, label = test_data[0][0], test_data[0][1]
with torch.no_grad():
    infer = model(data)
    # infer는 [batch_size, logits] 차원의 데이터가 된다
    infered, actual = classes[infer[0].argmax(0)],classes[label]
    print(f"infered data:{infered}, actual data:{actual}")

model successfully loaded
infered data:Ankle boot, actual data:Ankle boot


## Inference for real-world data

In [ ]:
# import torchvision.transforms as transforms
# from PIL import Image
# # Define transformations for real-world data
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize to the input size of your model
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet mean and std
# ])

# # Load and preprocess the real-world image
# image_path = 'real_world_image.jpg'
# image = Image.open(image_path)
# input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

## Basic Deeplearning

**Logistic Regression ( binary label regression )**

zsum = (parameters)W dot (input vector)X + w0(bias)

zsum 이 크고 작음을 통해 input을 분류한다

> zsum이 크다는 것? -> activation function (sigmoid etc..) 을 통해 클래스에 속할 확률을 계산한다. 

> sigmoid와 같은 활성홤수는 binary classes이기에 하나의 확률값(이 때 특정 확률값을 decision boundary로 설정 가능->hyperparam)만 return하면 된다. **또한 미분이 가능하다**

**loss function ( y^(예측값)과 y(label)사이 distance(loss,cost)를 return) loss function은 정답예측값에 대해서는 낮은 값, 틀린예측값에 대해서는 높은 값을 return 해야 된다.**

> cross entropy loss , 파라미터별 예측값(확률 p(y|x)의 negative log), 즉 -log(p(y|x))를 minimize하는 parameter를 채택해야된다.

> loss function은 인자(parameters)에 따라 값이 변하는 convex(아래로 볼록)한 함수로 minimum값이 존재한다.


**gradient descent는 back propagate(역전파)하며 층별로 gradient를 계산하는 것이다.**

> stochastic gradient descent 는 전체 데이터셋에서 일부 샘플(mini batch)을 채취해 학습에 사용 (gradient의 잡음을 줄여 수렴이 빠르다), 이 때 batch의 gradient 평균을 gradient로 사용

> gradient는 parameter 차원에 따른 벡터로 계산되어지고 벡터 차원별로 parameter가 갱신된다

> backpropagation후 step(갱신)단계에서 loss function의 gradient가 0인 지점인 loss의 minimum지점을 목표로 현재 인자(parameter)의 gradient 반대방향으로 인자를 진행시킨다, (parameter로부터 -gradient(기울기의 반대방향)을 더해가며 인자값을 조정한다.)

>진행률은 learning rate라는 hyper parameter를 gradient에 곱해주어 정한다.

**hyperparameter는 학습되어지는 것이 아니라 디자이너가 고르는 것이다.**

> 은닉층 노드 개수(hidden layer input channel) 또한 hyperparameter, learning rate, epoch, threshold, loss function 종류, optimzier 종류 등등

**step(갱신) 단계에서 Regularization과 dropout으로 overfit을 피하자**

> overfit된 가중치 -> 큰 가중치를 penalizes하는 @*R(^)를 빼준다, L2 R-> the sum of the square of the weights(Euclidean dist) L1 R -> the sum of the absolute value of the weight (Manhattan dist)

**multinomial Logistic Regression**

> y^의 부류별 합은 1 -> z = [z1,z2,z3] ,이 때 activation function으로 softmax를 취해주면 softmax(z)는 각 부류별 비율로 벡터를 정규화 시켜주고 max값을 예측값으로 취한다.


## Convolution Neural Network